# General instructions for all labs

1. To turn in:
 - this python notebook, filled out (2 pts)
 - a *standalone* PDF report that contains all the plots, and the answers to all the discussion questions (2 pts)

2. Use of ChatGPT / CoPilot / etc:
   - Allowed, but you own everything that is generated
   - This means that any part of the solution can be asked in the quiz. It can be as detailed as "What was the batch size you used in training" or specific as "what exactly does masking do in this case?" Any discussion question is also game for a quiz question.
   - If I find AI usage to be excessive. I can individually drag any of you in for a 1-1 meeting, in which I grill you on your code. If it looks like irresponsible copy/pasting, without proper understanding, I reserve the right to drastically lower your grade, or even submit cases to GGAC for ethical review.
  
3. Use of peer collaboration:
   - In general not allowed. (Discussion / comparing answers is ok, but work on actual coding independently.)
   - Exceptions can be made if you all wrote your own training script, but 1. it takes forever to train or 2. you don't have great compute resources. Then you can share a trained model amongst yourself *and declare it on your pdf*. However, the code for training *still must be written by yourself*
     


## **Lab 1 Overview**

This lab explores how domain-specific training shapes NanoGPT’s internal representations and output behavior. You will train three separate *small* NanoGPT models on distinct, **tiny** corpora (each \~50–200 KB of text):

1. **Shakespeare corpus** – Short subset of public-domain plays.
2. **Wikipedia corpus** – Small scraped/cleaned sample from open Wikipedia dumps.
3. **Math textbook corpus** – Small public-domain math primer or textbook excerpt.

By comparing these models across zero-shot and few-shot settings, you will observe how the same generative architecture adapts its “classification” over the vocabulary depending on the training distribution.


## **Learning Outcomes**

By the end of the lab, students should:

1. Recognize that **generation is token-level classification**, and that domain-specific data shifts the learned classification boundaries.
2. Understand **zero-shot vs. few-shot** generalization in the context of specialized generative models.
3. Interpret **softmax confidence** and detect overconfidence/oversaturation phenomena.
4. Apply **model interpretability tools** (Grad-CAM) to compare how domain training changes token importance.
5. Connect all observations to **overfitting, evaluation metrics, and embeddings**.


## **Part 1 – Training Domain-Specific Models**

* Use the **same small NanoGPT config** (e.g., `n_layer=2`, `n_head=2`, `n_embd=128`, `block_size=64`) for all runs.
* Train on CPU or free Colab GPU — each model should converge in minutes given the small corpora and reduced parameters.
* Monitor:

  * Training loss curves.
  * Qualitative performance of generated samples.

This section mostly requires you to run scripts that are already written. At each step, pay close attention to the comments, as you may be asked about them in the future. 

In [ ]:
import os, io, zipfile, requests
from pathlib import Path

# Choose the corpus
corpus_name = "shakespeare"  # Change to "shakespeare", "wikipedia", or "math"

# Reliable sources:
# - shakespeare: Karpathy's tiny Shakespeare (plain text)
# - wikipedia: https://www.kaggle.com/datasets/ffatty/plain-text-wikipedia-simpleenglish
# - math: https://archive.org/stream/CalculusMadeEasy/Calculus_Made_Easy_Thompson_djvu.txt
 


# Download and show sample
 
text = Path(f"data/{corpus_name}.txt").read_text(encoding="utf-8", errors="ignore")
print(f"Corpus length: {len(text)} characters")
print("Sample:")
print(text[:300])


In [ ]:
# -----------------------------------------
# Prepare dataset for character-level modeling
# -----------------------------------------

import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from torch.utils.data import DataLoader

class CharDataset(Dataset):
    def __init__(self, text, block_size, stoi=None, itos=None):
        """
        text: The raw text string we want to train on.
        block_size: The length of each training sequence (number of characters).
        stoi, itos: Optional vocab mappings. If provided, reuse them.
        """

        # 1. Build or reuse the vocabulary
        if stoi is None or itos is None:
            # Build from scratch
            self.chars = sorted(list(set(text)))
            self.vocab_size = len(self.chars)
            self.stoi = {ch: i for i, ch in enumerate(self.chars)}
            self.itos = {i: ch for i, ch in enumerate(self.chars)}
        else:
            # Reuse given vocab
            self.stoi = stoi
            self.itos = itos
            self.vocab_size = len(self.stoi)

        # 2. Store sequence length
        self.block_size = block_size

        # 3. Encode dataset into indices
        #    Use .get(ch, 0) so unknown characters map to 0
        self.data = torch.tensor([self.stoi.get(c, 0) for c in text], dtype=torch.long)

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        chunk = self.data[idx : idx + self.block_size + 1]
        x = chunk[:-1]
        y = chunk[1:]
        return x, y


block_size = 64
text = Path(f"data/{corpus_name}.txt").read_text(encoding="utf-8", errors="ignore")
dataset = CharDataset(text, block_size=block_size, stoi=None, itos=None)
 
 


 
## Analysis: Character Frequency Histogram

Visualize the differences in vocabulary and character usage between corpora (`shakespeare`, `wikipedia`, `math`). Do this by creating a histogram of each character's frequency of occurance, for each corpora.
  
Unicode safety

   * Some characters may not render properly in plots or Jupyter.
   * Replace problematic characters with their Unicode code point.
   * Example helper function:

     ```python
     def safe_label(c):
         try:
             c.encode("ascii")  # check if ASCII-printable
             return c
         except UnicodeEncodeError:
             return f"U+{ord(c):04X}"
     ```

 
   * Generate one histogram for each corpus (`shakespeare`, `wikipedia`, `math`).
   * For each corpus, write **1–2 sentences** describing:

     * Which characters are most common.
     * Any unusual symbols or formatting.
     * How the distributation of the corpus represents a distribution of a more general language model, e.g. one learned by ChatGPT.



In [ ]:
from nanogpt_model import GPT  # Import the GPT model class from the NanoGPT repo


# -----------------------------------------
# Define a configuration object for the GPT model
# -----------------------------------------
class GPTConfig:
    def __init__(self, vocab_size, block_size,
                 n_layer=2, n_head=2, n_embd=128, dropout=0.0, bias = True):
        """
        This class is a container for all the key hyperparameters
        that define the size and shape of the GPT model.
        We pass this config into the GPT constructor so the model
        can be built with these exact settings.

        ---------------------------
        Parameter meanings:
        ---------------------------

        vocab_size:
            - Number of unique tokens in our dataset.
            - In a character-level model, this is the number of distinct characters.
            - Maps directly to the size of the token embedding matrix:
                token_embedding_table.shape == (vocab_size, n_embd)

        block_size:
            - Maximum context length (sequence length) the model sees at once.
            - Sets the width of the positional embedding table:
                position_embedding_table.shape == (block_size, n_embd)
            - Also defines the mask size in self-attention so the model only attends
              to the last `block_size` tokens.

        n_layer:
            - Number of Transformer blocks stacked in the model.
            - Each block = (Multi-Head Self-Attention + Feedforward MLP + LayerNorm).
            - More layers allow the model to capture more complex dependencies.

        n_head:
            - Number of attention heads per multi-head attention layer.
            - Each head learns to focus on different positions/tokens in the sequence.
            - Heads are concatenated then projected back into `n_embd` dimensions.

        n_embd:
            - Dimensionality of token embeddings and all hidden states.
            - Controls the "width" of the model.
            - Affects:
                * Token embedding size
                * Positional embedding size
                * Per-head dimension in attention: head_dim = n_embd / n_head
                * Hidden size of the feedforward layers in each block.

        dropout:
            - Dropout probability applied in various places (attention, MLP) during training.
            - Helps prevent overfitting by randomly zeroing activations.
 
        """

        # Store all parameters for use by the GPT class
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.n_layer = n_layer
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout 
        self.bias = bias  


config = GPTConfig(vocab_size=dataset.vocab_size, block_size=block_size)
model = GPT(config)
print(model)

 

 

## Analysis: Model Architecture Diagram
 
Create a clear, labeled diagram of the NanoGPT model you trained, showing the **flow of data** from input to output, including the **shapes/dimensions** at each stage. Do this *by hand*. You may use a tablet, but it cannot look code-generated. 

 
1. **Overall Layout**

   * Show the full pipeline:
     **Input Tokens → Token Embedding → Positional Embedding → Transformer Blocks → LayerNorm → Output Head → Softmax**.

2. **Transformer Blocks**

   * Each block should include:

     * **LayerNorm 1 → Multi-Head Causal Self-Attention → Residual Connection**
     * **LayerNorm 2 → Feedforward MLP → Residual Connection**
   * Clearly label these sub-components and indicate that this block repeats `n_layer` times.

3. **Dimensions**

   * Label the **shape** of the tensor at each stage (batch size = `B`, sequence length = `T`, embedding size = `n_embd`):

     * Input: `(B, T)` (integer token IDs)
     * Token embeddings: `(B, T, n_embd)`
     * After adding positional embeddings: `(B, T, n_embd)`
     * Inside attention: queries/keys/values → `(B, n_head, T, head_dim)`
     * MLP layers: `(B, T, 4*n_embd)` then back to `(B, T, n_embd)`
     * Output logits: `(B, T, vocab_size)`

4. **Connections**

   * Draw arrows between each component to show the data flow.
   * Mark residual connections that add the block’s input to its output.

5. **Label Hyperparameters**

   * Include:

     * `n_layer` = number of transformer blocks
     * `n_head` = number of attention heads per block
     * `n_embd` = embedding dimension
     * `block_size` = maximum context length
     * `vocab_size` = size of the token vocabulary

6. **Final Output**

   * Show that the model outputs logits for each token position, then a softmax over the vocabulary to get probabilities.

 
 

 

## Training NanoGPT

The following code actually **trains NanoGPT** on your selected dataset.
Your job is to:

1. **Play around with the hyperparameters** (e.g. learning rate, batch size, context length, number of layers).
2. **Run the training loop** long enough to get meaningful results.
3. **Save a trained checkpoint for each of the three corpora** (`shakespeare`, `wikipedia`, `math`).
 
After training:

* You should have a folder of checkpoints (e.g. `checkpoints/shakespeare/final.pt`).
* Each corpus will give you a slightly different “voice” when you generate text.
 

In [ ]:
import os
import torch
from tqdm import tqdm
print(corpus_name)
# Directory for this corpus
ckpt_dir = os.path.join("checkpoints", corpus_name)
os.makedirs(ckpt_dir, exist_ok=True)
final_ckpt_path = os.path.join(ckpt_dir, "final.pt")

if os.path.exists(final_ckpt_path):
    print(f"Final checkpoint for '{corpus_name}' found at {final_ckpt_path}.")
    ckpt = torch.load(final_ckpt_path, map_location="cpu")
    model.load_state_dict(ckpt["model_state"])
    losses = ckpt.get("losses", [])
    print(f"Loaded model with {len(losses)} stored loss values.")
else:
    print(f"No checkpoint found for '{corpus_name}', starting training...")
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    max_iters = 5000

    losses = []   # will now store only full-batch eval losses
    model.train()
    for it in range(max_iters):
        xb, yb = next(iter(loader))

        logits, loss = model(xb, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if it % 100 == 0:
            # Evaluate on full dataset (avg loss across all batches)
            model.eval()
            eval_losses = []
            
            max_batches = len(loader) * 0.01
            for i, (xb_eval, yb_eval) in enumerate(loader):
                if i >= max_batches:  break
                logits_eval, loss_eval = model(xb_eval, yb_eval)
                eval_losses.append(loss_eval.item())
            avg_loss = torch.mean(torch.tensor(eval_losses))
            losses.append(avg_loss)
            print(f"Iter {it}/{max_iters}, Avg Loss over dataset: {avg_loss:.4f}")

            # Save periodic checkpoint
            iter_ckpt_path = os.path.join(ckpt_dir, f"iter_{it}.pt")
            torch.save({
                "model_state": model.state_dict(),
                "config": config.__dict__,
                "itos": dataset.itos,
                "stoi": dataset.stoi,
                "losses": losses
            }, iter_ckpt_path)
            print(f"Saved checkpoint: {iter_ckpt_path}")
            model.train()  # back to training mode

    # Save final checkpoint
    torch.save({
        "model_state": model.state_dict(),
        "config": config.__dict__,
        "itos": dataset.itos,
        "stoi": dataset.stoi,
        "losses": losses
    }, final_ckpt_path)
    print(f"Saved final model to {final_ckpt_path}")


## Analysis: Track Model Output Over Training


### **Procedure**
* Extract model checkpoints every **100 iterations** (e.g., `iter_100.pt`, `iter_200.pt`, `iter_300.pt`, …).
* For each corpus (`shakespeare`, `wikipedia`, `math`):
 * Use the provided generation code to produce **one line of generated text** from the **same fixed prompt** (e.g., `"Once upon a time"`).
 * If you find the output has not yet achieved the langauge structure quality you desire, you may want to train it for longer. Go until you have a strong model.

---

### **Consistency**

* Keep the **prompt** identical across all checkpoints, but consistent to the corpora in mind.
* Use the same **number of generated tokens** each time.
* This ensures you can make direct comparisons between outputs.

---

### **Reporting**

In your lab report:

* Include a short **table or list** for each corpus showing:

  * Iteration number.
  * The generated output (first \~100 characters).

* Discuss the following:

  * At what point does the text start to look coherent?
  * How does the style differ between Shakespeare, Wikipedia, and Math?
  * Does the model become **overconfident** (repetitive text, fixed patterns) at later stages?

In [ ]:
import os

# This code loads a specific training checkpoint of NanoGPT
# (saved during training) and uses it to generate text.


def generate_small_text(corpus, start_text, iteration_to_load, max_new_tokens):
    # Path to the specific checkpoint
    
    if iteration_to_load == "final":
        ckpt_path = os.path.join("checkpoints", corpus, "final.pt")
    else:
        ckpt_path = os.path.join("checkpoints", corpus, f"iter_{iteration_to_load}.pt")
    
    if not os.path.exists(ckpt_path):
        print(f"Checkpoint {ckpt_path} not found. Make sure to train first.")
    else:
        # Load checkpoint from disk
        ckpt = torch.load(ckpt_path, map_location="cpu")
    
        # Rebuild the GPT model from saved config
        loaded_config = type("GPTConfig", (), ckpt["config"])
        gen_model = GPT(loaded_config)
        gen_model.load_state_dict(ckpt["model_state"])
        gen_model.eval()
    
        # Tokenization: convert characters to IDs
        stoi = ckpt["stoi"]
        itos = ckpt["itos"]
        idx = torch.tensor([stoi.get(c, 0) for c in start_text], dtype=torch.long).unsqueeze(0)
    
        # Autoregressive generation loop
        with torch.no_grad():
            for _ in range(max_new_tokens):
                # Feed only the last block_size tokens
                idx_cond = idx[:, -loaded_config.block_size:]
                logits, _ = gen_model(idx_cond)
    
                # Convert logits to probabilities
                probs = torch.softmax(logits[:, -1, :], dim=-1)
    
                # Sample next token from probability distribution
                idx_next = torch.multinomial(probs, num_samples=1)
    
                # Append new token to the sequence
                idx = torch.cat((idx, idx_next), dim=1)
    
        # Decode IDs back into text
        generated_text = ''.join([itos[i.item()] for i in idx[0]])
        return generated_text

## Analysis: Variability of the Final Model

### **Procedure**

* After training, take the **final checkpoint** for each corpus.
* Using the same prompt as before (corpus-consistent), generate **multiple outputs (at least 5)**.
* Each output should use the same number of tokens, so they are directly comparable.

---

### **Consistency**

* Keep the prompt identical across all generations for each corpus.
* Do not change model parameters, only re-run generation.

---

### **Reporting**

In your lab report:

* Include the set of generated outputs from the final model.
* Comment on:

  * The **variance** across outputs: do they differ in interesting ways, or are they repetitive?
  * Whether the variability seems to capture the **style** of the corpus.
  * Whether you are satisfied with the outputs: does it feel like the model “learned” something meaningful?





## **Analysis: Plot Training Loss from Checkpoints**

* Produce **one loss curve plot per corpus** (`shakespeare`, `wikipedia`, `math`).
* Compare and contrast the training behavior across the different corpora.

**What to Comment On**

* Identify which loss values seem to correspond to **qualitatively good performance** (e.g., when the model starts producing coherent, domain-appropriate text).
* Note that the **initial drop in loss** is often not enough: real quality emerges only once the model reaches **very low error**.
* Discuss differences across corpora:

  * Which corpus needed lower loss values before outputs became convincing?
  * Do some domains “sound good” ons for what to hand in)?


## **Part 2 – Cross-Domain Evaluation**

* Evaluate each trained model **on each other’s dataset**:

  * **Zero-shot**: Direct prompt without domain examples.
  * **Few-shot**: Include 1–3 in-domain examples in the prompt.
* Compare:

  * Loss in-domain vs. out-of-domain.
  * Quality and relevance of generated continuations.





# Detailed Instructions

### **Part 2 Step 1: Zero-shot Evaluation**

For each dataset pair (A, B) where A, B ∈ {`wikipedia`, `math`, `shakespeare`}:

1. Load the **final checkpoint** of the model trained on A.

   * When rebuilding the model, you must restore the **vocab size** from the checkpoint, e.g.

     ```python
     vocab_size = len(ckpt["stoi"])
     model = GPT(vocab_size=vocab_size)
     ```
   * This ensures the embedding and output layers match the original training.
2. Evaluate it on dataset B **without in-domain examples**. Do so on 1% of the training data, as it is very large.
3. Record the **loss values** in a 3×3 table (diagonal = in-domain).
4. For each pair, include one **generated sample** with these instructions?


### **Part 2 Step 2: Few-shot Evaluation**

1. **Start from a Pretrained Model (Dataset A).**

   * Choose a model that has already been trained on dataset **A** (e.g., `wikipedia`, `math`, `shakespeare`).
   * Load the **final checkpoint** for A:

     ```python
     ckpt_path = f"checkpoints/{A}/final.pt"
     ckpt = torch.load(ckpt_path, map_location="cpu")
     ```
   * Important: the vocabulary size used for training A may differ from the default.

     * Before building the model, set

       ```python
       config.vocab_size = len(ckpt["stoi"])
       model = GPT(config)
       ```
     * This ensures the embedding and output layer dimensions match the checkpoint.
   * Finally, load the weights:

     ```python
     model.load_state_dict(ckpt["model_state"])
     ```

2. **Fine-tune on Dataset B.**

   * Use the **same vocabulary (`stoi`, `itos`) from dataset A** to enco0de dataset B.
   * Train for **200 iterations** with AdamW (`lr=**10, 25, 50, 100, 2ns thereafter if you extend training.
   * Naming convention:

     ```
     {A}_start{N}_{B}_run{M}.pt
     ```

     where

     * **A** = source dataset (e.g. `wikipedia`)
     * **B** = target dataset (e.g. `math`)
     * **N** = starting iteration (e.g. `0` if from scratch, or `500` if resuming)
     * **M** = current fine-tuning iteration

3. **Evaluate and Track Loss.**

   * Every 50 steps, evaluate on a **1% subsample of dataset B** for speed.
   * Record the average evaluation loss.
   * Keep a running list of these evaluation losses to plot later.

4. **Plot Loss Curves.**

   * After training, plot **loss vs iteration** for each experiment.
   * Compare curves across different (A→B) fine-tuning runs.
   * This shows how quickly and effectively each pretrained model adapts to dataset B.

5. **Examples**

   * At each saved checkpoint (10, 25, 50, 100, 200), generate short text continuations from the model.
   * Use the same sampling routine (temperature-controlled decoding).
   * Collect and compare generations to see how quality improves over epochs



## **Part 2 Step 3: Few-shot from Earlier Checkpoints**

* Repeat Step 2, but start from models that were **not fully trained** on A.
* Compare results: does partial training make adaptation easier? 

### **Deliverables**

* **Tables**: Zero-shot loss values (3×3).
* **Plots**: Few-shot loss curves (per experiment).
* **Samples**: Generated text at selected iterations.
* **Discussion**: Briefly how to structure their zero-shot results?


## **Part 3 – Inspecting the Softmax Output Layer**

In this part, you will explore how the model’s **output probabilities** evolve during training for different prompts.

### **What to Do**

* Choose **several prompts** (at least 2–3) that differ in domain and style (e.g., math expressions, natural language, biology terms).
* For each prompt, extract at each checkpoint:

  * The **softmax probability distribution** over the vocabulary.
  * The **entropy** of the distribution (a measure of model confidence/uncertainty).
* Plot the **trajectories of top candidate tokens** over checkpoints for each prompt.

### **What to Look For**

* **Predictability & Hardening**

  * Does the distribution concentrate (one token dominates) as training progresses?
  * Are some prompts much more predictable than others?

* **Entropy & Uncertainty**

  * Compare entropy across prompts.
  * Do in-domain prompts (e.g., math-related text) produce sharper, lower-entropy distributions?
  * Do out-of-domain prompts produce flatter, high-entropy distributions?

* **Biases in Token Predictions**

  * Are certain tokens consistently high-probability regardless of prompt?
  * Do different prompts encourage different sets of top tokens?

### **What to Report**

* Provide **plots** for each prompt.
* Give a **written description** of what you observe:

  * Where do probabilities harden?
  * Where do they remain diffuse?
  * How does this match your expectations about the model’s training data?
* Discuss what your findings imply about the model’s **predictability and specialization**.


In [ ]:
import torch
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

def load_model(ckpt_path,  GPT):
    ckpt = torch.load(ckpt_path, map_location="cpu")
    config = type("GPTConfig", (), ckpt["config"])
    stoi, itos = ckpt["stoi"], ckpt["itos"]
    config.vocab_size = len(stoi)
    model = GPT(config)
    model.load_state_dict(ckpt["model_state"])
    return model, stoi, itos

def inspect_softmax(model, prompt, stoi, itos, top_k=10):
    # pick a fallback index if <unk> not in vocab
    unk_idx = stoi.get("<unk>", next(iter(stoi.values())))  

    x = torch.tensor([[stoi.get(c, unk_idx) for c in prompt]], dtype=torch.long)
    with torch.no_grad():
        probs = F.softmax(model(x)[0][:, -1, :], dim=-1).squeeze()
    entropy = -(probs * torch.log(probs + 1e-12)).sum().item()
    top_p, top_i = torch.topk(probs, top_k)
    return {
        "prompt": prompt,
        "entropy": entropy,
        "probs": [(itos[i.item()], p.item()) for i, p in zip(top_i, top_p)]
    }


 
## **Part 4 – Grad-CAM for Language Models**

In this part, you’ll build a lightweight version of *Grad-CAM* for a language model. Instead of visualizing gradients in images, we’ll look at how gradients flow into **token embeddings** to measure which input tokens most affect a model’s probability of generating a target token.

### **Steps**

1. **Compute gradients**

   * Use the provided `token_gradients` function.
   * This function backpropagates from the probability of a **target token** (e.g., `"e"`) back to the input embeddings.
   * For each input token, it returns the gradient norm, which indicates **how much changing that token would affect the prediction**.
 
2. **Experiment with checkpoints**

   * Run the method across checkpoints of your trained model (e.g., every 100 iterations).
   * Track how gradient importance changes over time.
   * Try different prompts and different target tokens.

3. **Compare across models**

   * Run the same analysis on domain-trained models (e.g., trained on Wikipedia vs. math text).
   * Compare: Do the same tokens consistently drive predictions? Or do models in different domains “care” about different tokens?

 
### **Discussion Questions**

1. **Character influence:**
   Which characters consistently receive the highest gradient norms? Are they vowels, consonants, punctuation, or digits? Do frequent letters (like “e”) dominate, or do rare characters sometimes matter more?
2. **Checkpoint dynamics:**
   How does the distribution of influential tokens change as training progresses? Does the model become more focused or more diffuse?
3. **Domain differences:**
   Compare results from different domain-trained models. Do Wikipedia-trained models rely more on content words, while math-trained ones emphasize structure/symbols?
4. **Zero-shot vs few-shot:**
   If you add example completions to the prompt (few-shot), how does the gradient map shift compared to zero-shot?
5. **Interpretability limits:**
   What are the limitations of gradient-based methods? Do high gradient norms always mean high “importance”? When might this method be misleading?
  


In [ ]:
import torch
import torch.nn.functional as F

import os
import matplotlib.pyplot as plt
import numpy as np

def token_gradients(model, prompt, target_token, stoi, itos ):
    """
    Compute gradient norms per input token wrt probability of `target_token`.
    """
    model.eval()
    x = torch.tensor([stoi[c] for c in prompt], dtype=torch.long)[None, :] 

    grads = {}

    # Hook to capture embeddings with gradient
    def save_grad(module, inp, out):
        out.retain_grad()
        grads['emb'] = out

    handle = model.transformer.wte.register_forward_hook(save_grad)

    # Forward pass
    logits, _ = model(x)
    last_logits = logits[:, -1, :]   # [B, vocab]

    target_idx = stoi.get(target_token, None)
    if target_idx is None:
        raise ValueError(f"Token {target_token!r} not in vocab")

    probs = F.softmax(last_logits, dim=-1)
    target_prob = probs[0, target_idx]

    # Backprop
    model.zero_grad(set_to_none=True)
    target_prob.backward()

    # Now grads['emb'] has the embedding grads
    grad_norms = grads['emb'].grad[0].norm(dim=-1)  # [T]
    tokens = [itos[i.item()] for i in x[0]]

    # Clean up the hook
    handle.remove()

    return list(zip(tokens, grad_norms.tolist()))

